In [ ]:
from collections import OrderedDict
import numpy as np
import tensorflow as tf

import gym
from kaggle_environments import make
import kaggle_environments.envs.halite.helpers as hh
from tf_reinforcement_testcases import deep_q_learning, misc
from gym_halite.envs.halite_env import get_scalar_features, get_feature_maps

In [ ]:
halite = "gym_halite:halite-v0"
cart_pole = "CartPole-v1"
env_name = halite
agent = deep_q_learning.PriorityDoubleDuelingDQNAgent(env_name)

In [ ]:
model = agent.train(iterations_number=20000)

In [ ]:
board_size = 5
starting_halite = 5000
env = make("halite",
           configuration={"size": board_size,
                          "startingHalite": starting_halite},
           debug=True)
obs = env.reset()
trainer = env.train([None])
obs = trainer.reset()
board = hh.Board(obs, env.configuration)

In [ ]:
skalar_features = get_scalar_features(board)
skalar_features = skalar_features[np.newaxis, ...]
skalar_features = tf.convert_to_tensor(skalar_features, dtype=tf.float32)
feature_maps = get_feature_maps(board)
feature_maps = feature_maps[np.newaxis, ...]
feature_maps = tf.convert_to_tensor(feature_maps, dtype=tf.float32)
obs = OrderedDict({'feature_maps': feature_maps, 'scalar_features': skalar_features})
Q_values = model(obs)
print(Q_values)

In [ ]:
if env_name == cart_pole:
    env = gym.make("CartPole-v1")
    observation = env.reset()
    for _ in range(1000):
        env.render()
        Q_values = model(observation[np.newaxis, :])
        observation, reward, done, info = env.step(np.argmax(Q_values.numpy()))

        if done:
            observation = env.reset()
    env.close()

In [ ]:
if env_name == halite:
    halite_agent = misc.get_halite_agent(model)
    # these are the settings used for a halite v0.1
    board_size = 5
    starting_halite = 5000

    env = make("halite",
               configuration={"size": board_size,
                              "startingHalite": starting_halite},
               debug=True)
    env.run([halite_agent])
    env.render(mode="ipython", width=800, height=600)